In [1]:
import pandas as pd
import numpy as np

!pip install transformers
!pip install sentencepiece
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.6 MB/s eta 0:00:00


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum').to(device)

In [3]:
device

'cuda'

In [4]:
data_dir = '/content/drive/MyDrive/Linguistic abstraction summer 2023/Datasets/Raw text data/'

In [39]:
div = pd.read_csv(data_dir+'diversity_study_text_130423_raw.csv', index_col=0)

In [8]:
pilot_ego = pd.read_csv('/content/pilot_ego_text_export_192_8.8.22 - pilot_ego_text_export_192_8.8.22.csv', index_col=0)

In [5]:
pilot_nyu = pd.read_csv('/content/pilot_NYU_text_export_192_8.8.22 - pilot_NYU_text_export_192_8.8.22.csv', index_col=0)

In [6]:
pilot_nyu.head()

,S1_ellab,S12_ellab,A1_ellab,A12_ellab
ID,,,,
1,I am currently a first-year student at NYU and...,NaN,NaN,Since coming to NYU I have been on campus more...
2,NaN,The social life in NYU is interesting. There a...,"I am majoring Media, Culture, Communication. T...",NaN
3,NaN,My social life so far has been pretty good. I ...,My academic life at NYU has been pretty rigoro...,NaN
4,I have meet a lot of friends after entering NY...,NaN,NaN,NYU has the greatest professors in the world. ...
5,NaN,NYU is a very diverse college with many opport...,NYU is a fun college to come to if you are ric...,NaN


In [18]:
pilot_ego.head()

,ES_ellab,EP_ellab,AS_ellab,AP_ellab
ID,,,,
1,"I imagine, that I was friend with Ariel. I ima...",I visited Sam and Jessie between classes at th...,Taylor ate with Ariel during their break betwe...,Taylor talked with Sam and Jessie as they wait...
2,I am eating with Sam. We are eating pancake at...,I am walking with Ariel and Taylor. I am in th...,Jessie standing in front of Sam's door.,Jessie is standing in front of Ariel and Taylor.
3,I picture the journey to Sam's house on a sunn...,We are walking down a sunny street with iced c...,"I imagine a similar scene, it being a sunny da...",On an outdoor patio outside a pizza parlor I s...
4,I talk about the exam. Sam told me that I got ...,I am having dinner with Jessie and Taylor. We ...,Ariel and Sam are good friends. They just had ...,Ariel might be a friend of Jessie and Taylor. ...
5,I ate pizza with Sam. Sam had a salad. We talk...,I went to Ariel's home to visit Jessie and Tay...,Ariel is breaking up with Sam. She wants to te...,"Ariel, Jessie, and Taylor are best friends. Th..."


In [17]:
div.head(3)

,hetero_text,homo_text
ID,,
1,We all have the same seminar class. Taylor is ...,We all grab lunch and coffee to discuss over c...
2,I imagine us in a group at some sort of social...,I imagine us sitting in Washington square park...
3,I picture them dispersed around the room where...,I would imagine those who have known each othe...


In [18]:
doc = div.iloc[0,0]
doc

'We all have the same seminar class. Taylor is the only freshman while majority of them are junior. No one was in the same page regarding the assignment. It took forever to decide on a topic but they finally picked The Slave Triangle. Taylor did most of the work because everyone else was busy which frustrated them a lot.'

In [44]:
inputs = tokenizer(doc, truncation=True, max_length=1024, return_tensors="pt").to(device)
summary_ids = model.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"Taylor's class had a problem with their homework."

In [59]:
hetero_summary = []
for i in range(div.shape[0]):
  txt = div.hetero_text.values[i]
  tokens = tokenizer(txt, truncation=True,  padding=True, max_length=1024, return_tensors="pt").to(device)
  summary_ids = model.generate(tokens['input_ids'])
  summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  hetero_summary.append(summary[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [65]:
homo_summary = []
for i in range(div.shape[0]):
  txt = div.homo_text.values[i]
  tokens = tokenizer(txt, truncation=True,  padding=True, max_length=1024, return_tensors="pt").to(device)
  summary_ids = model.generate(tokens['input_ids'])
  summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  homo_summary.append(summary[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [63]:
div['hetero_summary_pegasus'] = hetero_summary
div['homo_summary_pegasus'] = homo_summary

In [67]:
div.head(3)

,hetero_text,homo_text,hetero_summary_pegasus,homo_summary_pegasus
ID,,,,
1,We all have the same seminar class. Taylor is ...,We all grab lunch and coffee to discuss over c...,Taylor's class had a problem with their homework.,On the first day of class we decide to do a pr...
2,I imagine us in a group at some sort of social...,I imagine us sitting in Washington square park...,How would you feel if you were from a differen...,What would you do if you were in charge of a g...
3,I picture them dispersed around the room where...,I would imagine those who have known each othe...,All images are copyrighted.,What would it be like to meet up with someone ...


In [54]:
pilot_nyu.apply(lambda c: c.apply(lambda x: 0 if type(x) is float else len(x))).mean()

S1_ellab     194.468750
S12_ellab    161.494792
A1_ellab     168.645833
A12_ellab    193.432292
dtype: float64

In [7]:
def generate_pegasus_summary(df, col):
  col_summary = []
  for i in range(df.shape[0]):
    txt = df[col].values[i]
    if type(txt) is float: 
      col_summary.append(float('nan'))
      continue
    tokens = tokenizer(txt, truncation=True,  padding=True, max_length=1024, return_tensors="pt").to(device)
    summary_ids = model.generate(tokens['input_ids'], max_new_tokens=30)
    summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    col_summary.append(summary[0])
  return col_summary

In [8]:
for col in pilot_nyu.columns:
  col_summary = generate_pegasus_summary(pilot_nyu, col)
  pilot_nyu[col+'_summary_pegasus'] = col_summary

In [9]:
pilot_nyu.head(3)

,S1_ellab,S12_ellab,A1_ellab,A12_ellab,S1_ellab_summary_pegasus,S12_ellab_summary_pegasus,A1_ellab_summary_pegasus,A12_ellab_summary_pegasus
ID,,,,,,,,
1,I am currently a first-year student at NYU and...,NaN,NaN,Since coming to NYU I have been on campus more...,How have you made friends at New York University?,NaN,NaN,"As a first-generation college student, I have ..."
2,NaN,The social life in NYU is interesting. There a...,"I am majoring Media, Culture, Communication. T...",NaN,NaN,New York University (NYU) is one of the most p...,I am a junior at New York University.,NaN
3,NaN,My social life so far has been pretty good. I ...,My academic life at NYU has been pretty rigoro...,NaN,NaN,I have been here for two months now and I have...,This is my second year at New York University.,NaN


In [10]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=7b4716b5eaa9845762b1c2e93c31f030471433aa51811b156a966209db86eb56
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [11]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

In [73]:
embeddings = div.apply(lambda c: c.apply(lambda x: model.encode(x))).to_numpy()

In [82]:
embeddings.shape

(331, 4, 384)

In [81]:
embeddings = np.hstack(embeddings.reshape(-1)).reshape(div.shape[0], div.shape[1], -1)

In [93]:
hetero_scores = np.array([cosine_similarity(embeddings[i,0,:].reshape(1,-1), embeddings[i,2,:].reshape(1,-1)) for i in range(div.shape[0])]).flatten()
homo_scores = np.array([cosine_similarity(embeddings[i,1,:].reshape(1,-1), embeddings[i,3,:].reshape(1,-1)) for i in range(div.shape[0])]).flatten()

In [97]:
div['hetero_pegasus_sim1'] = hetero_scores
div['homo_pegasus_sim1'] = homo_scores

In [33]:
def similarity_score(df):
  embeddings = df.iloc[:,:8].apply(lambda c: c.apply(lambda x: x if type(x) is float else model.encode(x))).to_numpy()
  #embeddings = np.hstack(embeddings.reshape(-1)).reshape(df.shape[0], df.shape[1], -1)
  for i,j in zip(range(0,4), range(4,8)):
    scores = np.array([float('nan') if type(embeddings[r,i]) is float else cosine_similarity(embeddings[r,i].reshape(1,-1), embeddings[r,j].reshape(1,-1)).flatten()[0] for r in range(df.shape[0])])
    df[df.columns[i]+'_pegasus_sim2'] = scores

In [27]:
np.array([float('nan') if type(e[r,0]) is float else cosine_similarity(e[r,0].reshape(1,-1), e[r,4].reshape(1,-1)).flatten()[0] for r in range(192)])

array([0.61110288,        nan,        nan, 0.66880333,        nan,
              nan,        nan, 0.7635144 ,        nan, 0.55866218,
              nan,        nan, 0.14335226, 0.70004213,        nan,
       0.19623503,        nan, 0.61976361, 0.05629068,        nan,
              nan,        nan, 0.6644007 , 0.25916871,        nan,
       0.47603595,        nan,        nan, 0.6907599 , 0.48290616,
       1.00000012, 0.43067166, 0.69256431, 0.24483243,        nan,
       0.79827613,        nan, 0.69516289, 0.0778151 , 0.52885246,
       0.39459535, 0.43887788, 0.45961767,        nan, 0.27875385,
              nan, 0.50472164,        nan, 0.74383152,        nan,
       0.30617756,        nan, 0.35176143, 0.62491262,        nan,
              nan, 0.70667112, 0.78284562, 0.72742081, 0.60911   ,
       0.25472903,        nan, 0.58453959, 0.52570379,        nan,
       0.38433436, 0.49631053, 0.83624893, 0.58257711, 0.7071209 ,
       0.57011604, 0.72058451,        nan,        nan, 0.35034

In [34]:
similarity_score(pilot_nyu)

In [35]:
pilot_nyu.head(3)

,S1_ellab,S12_ellab,A1_ellab,A12_ellab,S1_ellab_summary_pegasus,S12_ellab_summary_pegasus,A1_ellab_summary_pegasus,A12_ellab_summary_pegasus,S1_ellab_pegasus_sim1,S12_ellab_pegasus_sim1,A1_ellab_pegasus_sim1,A12_ellab_pegasus_sim1,S1_ellab_pegasus_sim2,S12_ellab_pegasus_sim2,A1_ellab_pegasus_sim2,A12_ellab_pegasus_sim2
ID,,,,,,,,,,,,,,,,
1,I am currently a first-year student at NYU and...,NaN,NaN,Since coming to NYU I have been on campus more...,How have you made friends at New York University?,NaN,NaN,"As a first-generation college student, I have ...",0.611103,NaN,NaN,0.338514,0.633481,NaN,NaN,0.437277
2,NaN,The social life in NYU is interesting. There a...,"I am majoring Media, Culture, Communication. T...",NaN,NaN,New York University (NYU) is one of the most p...,I am a junior at New York University.,NaN,NaN,0.725998,0.52092,NaN,NaN,0.646381,0.576214,NaN
3,NaN,My social life so far has been pretty good. I ...,My academic life at NYU has been pretty rigoro...,NaN,NaN,I have been here for two months now and I have...,This is my second year at New York University.,NaN,NaN,0.438068,0.55311,NaN,NaN,0.511859,0.602282,NaN


In [37]:
pilot_nyu.to_csv('drive/MyDrive/pilot_nyu_pegasus_results.csv')

In [32]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [103]:
embeddings = div.iloc[:,:4].apply(lambda c: c.apply(lambda x: model.encode(x))).to_numpy()
embeddings = np.hstack(embeddings.reshape(-1)).reshape(div.shape[0], 4, -1)

In [104]:
hetero_scores = np.array([cosine_similarity(embeddings[i,0,:].reshape(1,-1), embeddings[i,2,:].reshape(1,-1)) for i in range(div.shape[0])]).flatten()
homo_scores = np.array([cosine_similarity(embeddings[i,1,:].reshape(1,-1), embeddings[i,3,:].reshape(1,-1)) for i in range(div.shape[0])]).flatten()

In [105]:
div['hetero_pegasus_sim2'] = hetero_scores
div['homo_pegasus_sim2'] = homo_scores

In [106]:
div.head(3)

,hetero_text,homo_text,hetero_summary_pegasus,homo_summary_pegasus,hetero_pegasus_sim1,homo_pegasus_sim1,hetero_pegasus_sim2,homo_pegasus_sim2
ID,,,,,,,,
1,We all have the same seminar class. Taylor is ...,We all grab lunch and coffee to discuss over c...,Taylor's class had a problem with their homework.,On the first day of class we decide to do a pr...,0.501683,0.520848,0.582207,0.597927
2,I imagine us in a group at some sort of social...,I imagine us sitting in Washington square park...,How would you feel if you were from a differen...,What would you do if you were in charge of a g...,0.193676,0.378675,0.377779,0.440978
3,I picture them dispersed around the room where...,I would imagine those who have known each othe...,All images are copyrighted.,What would it be like to meet up with someone ...,0.102204,0.439804,-0.017377,0.401407


In [108]:
div.to_csv('/content/drive/MyDrive/div_pegasus_results.csv')